<strong><b><font size="5">Data Science Academy</font></b></strong>

<strong><b><font size="5">Web Scraping e Análise de Dados</font></b></strong>

<strong><b><font size="5">Lab 6</font></b></strong>

<strong><b><font size="4">Web Scraping e Análise de Preços e Configurações de Notebooks Para Data Science</font></b></strong>

![title](imagens/lab6.png)

## Definição do Problema

Procurando por um notebook novo para as aulas de Ciência de Dados na DSA? Pensando em comprar um novo computador para começar a empreender desenvolvendo aplicações analíticas? Sem tempo para ficar pesquisando pelas inúmeras opções disponíveis na internet?

Por que não criar seu próprio Web Scraper, que vai buscar, comparar e analisar diversas opções de notebooks para você e listar em uma tabela para sua escolha com os filtros que você especificar?

É o que faremos agora neste Lab.

Com a evolução da internet, o comércio eletrônico se tornou tão popular que agora praticamente todo mundo compra um ou outro produto online. Com apenas alguns cliques, seu produto desejado pode ser entregue na sua porta. Apesar de tão conveniente, às vezes pode ser exasperante, principalmente devido às inúmeras alternativas disponíveis em várias plataformas de comércio eletrônico.

Com este Lab vamos percorrer o site da Amazon e buscar detalhes sobre notebooks: preço, avaliação de clientes e configuração de hardware. Vamos limpar os dados, analisar e então resumir em uma tabela tudo que você precisa para tomar sua decisão.

Nós faremos Web Scraping na Amazon americana e como exercício deixaremos para você o ajuste no Lab para captura dos dados na Amazon brasileira. 

E lembre-se que temos um post sobre o tema em nosso Blog:

<a href="http://datascienceacademy.com.br/blog/qual-o-melhor-computador-pessoal-para-aprender-data-science/">Qual o melhor computador pessoal para estudar Data Science?</a>

Leia todos os comentários e execute as células.

## Fonte de Dados

Nossa fonte de dados será o site oficial da Amazon: https://amazon.com

## Carregando os Pacotes Usados Neste Jupyter Notebook

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.4


In [3]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook, se necessário.

In [2]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [4]:
# Imports
import bs4
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

requests 2.22.0
bs4      4.8.0
pandas   0.25.1
Data Science Academy


## Web Scraping

O primeiro passo é você acessar o endereço abaixo:

https://www.amazon.com/s?k=laptops

Esse endereço é o resultado da busca por "laptops". Inspecione o código fonte da página conforme você aprendeu no inicio do curso.

Coletaremos as informações relevantes das primeiras cinco páginas do resultado da pesquisa, pois isso nos fornecerá dados suficientes para a análise. Criaremos então um dataframe a partir dos dados coletados. 

Vamos criar uma função para cada ação de web scraping e depois juntar tudo em um loop como se fosse um roteiro. Preste atenção nos nomes das funções.

In [7]:
# Esta função define o cabeçalho da conexão ao site da Amazon e faz a requisição para extrair os dados
def faz_request(num_page):
    
    # Cabeçalho para evitar problema de conexão não segura ao site da Amazon
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", 
               "Accept-Encoding":"gzip, deflate", 
               "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
               "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    # Request para cada página do resultado de pesquisa
    # Observe o uso da variável num_page
    requisicao = requests.get('https://www.amazon.com/s?k=laptops&page='+str(num_page), headers = headers)
    
    return requisicao

In [8]:
# Função para extrair o conteúdo
def extrai_conteudo(requisicao):
    return requisicao.content

In [9]:
# Função para o parse do conteúdo extraído do site da Amazon
def formata_conteudo(conteudo):
    return BeautifulSoup(conteudo, 'lxml')

In [10]:
# Função para extrair o nome do notebook
# Visite o código fonte no site da Amazon para verificar a classe que contém o nome
# Você aprendeu como fazer isso no início do curso aqui na Data Science Academy
def extrai_nome(div):
    
    # Busca pela tag span, classe 'a-size-medium a-color-base a-text-normal', onde está o nome do notebook
    span_nome = div.find('span', attrs = {'class':'a-size-medium a-color-base a-text-normal'})
    
    # Se o valor não for vazio, extrai o valor (nome)
    if span_nome is not None:
        return span_nome.text
    else:
        return 'no-info' 

In [11]:
# Função para extrair o preço
def extrai_preco(div):
    
    # Busca pela tag span, classe 'a-offscreen', onde está o preço do notebook
    span_preco = div.find('span', attrs = {'class':'a-offscreen'})
    
    # Se o valor não for vazio, extrai o valor (preço)
    if span_preco is not None:
        return span_preco.text
    else:
        return 'no-info'

In [12]:
# Função para extrair a avaliação dos clientes
def extrai_aval(div):
    
    # Busca pela tag span, classe 'a-icon-alt', onde está a avaliação do notebook
    span_aval = div.find('span', attrs = {'class':'a-icon-alt'})
    
    # Se o valor não for vazio, extrai o valor (avaliação)
    if span_aval is not None:
        return span_aval.text
    else:
        return 'no-info'  

In [13]:
# Lista para receber os dados
dados_notebooks = []

In [14]:
# Loop por cada página retornada no scraping

print("\nExecutando Web Scraping...\n")

# Loop
for num_page in range(1,6):
    
    # Faz a request
    requisicao = faz_request(num_page)
    
    # Exrai o conteúdo
    conteudo = extrai_conteudo(requisicao)
    
    # Formata o conteúdo
    soup = formata_conteudo(conteudo)   
    
    # Loop pelo conteúdo
    for info in soup.findAll('div', attrs = {'class':'sg-col-4-of-12 sg-col-8-of-16 sg-col-16-of-24 sg-col-12-of-20 sg-col-24-of-32 sg-col sg-col-28-of-36 sg-col-20-of-28'}):
        
        # Extrai o nome do notebook
        nome = extrai_nome(info)
        
        # Extrai o preço do notebook
        preco = extrai_preco(info)
        
        # Extrai a avaliação do notebook
        aval = extrai_aval(info)
        
        # Adiciona os dados à lista
        dados_notebooks.append([nome, preco, aval])

print("\nExtração Concluída.\n")


Executando Web Scraping...


Extração Concluída.



Extração concluída. Temos uma lista com os dados. Vamos transformá-la em um dataframe do pandas para facilitar a manipulação dos dados.

In [15]:
# Cria dataframe
df_dados_notebooks = pd.DataFrame(dados_notebooks, columns = ['Nome', 'Preco', 'Avaliacao'])    

In [16]:
# Shape
df_dados_notebooks.shape

(16, 3)

In [17]:
# Visualiza os dados
df_dados_notebooks.head()

,Nome,Preco,Avaliacao
0,"2020 HP Pavilion 14"" HD Display Laptop Compute...",$339.00,4.2 out of 5 stars
1,HP 14in High Performance Laptop (AMD Ryzen 3 3...,$303.00,3.8 out of 5 stars
2,"ASUS ZenBook 13 Ultra Slim Laptop, 13.3” FHD W...",$949.99,4.0 out of 5 stars
3,2020 Lenovo IdeaPad Laptop ComputerAMD A6-9220...,$242.00,3.5 out of 5 stars
4,The Marvelous Mrs. Maisel - Season 2,no-info,4.2 out of 5 stars


In [18]:
# Visualizando os tipos de dados
df_dados_notebooks.dtypes

Nome         object
Preco        object
Avaliacao    object
dtype: object

Perfeito. Agora precisamos limpar os dados. Vamos à faxina.

## Limpeza da Coluna de Preços

Começaremos a limpeza pela coluna de preços. Leia os comentários com atenção.

In [19]:
# Buscando o total de registros por preço
df_dados_notebooks['Preco'].value_counts(dropna = False)

no-info    3
$682.00    1
$249.00    1
$242.00    1
$634.99    1
$949.99    1
$303.00    1
$588.00    1
$219.99    1
$339.00    1
$579.99    1
$638.99    1
$249.43    1
$355.98    1
Name: Preco, dtype: int64

In [20]:
# Temos vários registros com preço = no_info
# Registro sem preço, não queremos. Vamos remover
df_dados_notebooks = df_dados_notebooks[df_dados_notebooks['Preco'] != 'no-info']

In [21]:
# Buscando o total de registros por preço
df_dados_notebooks['Preco'].value_counts(dropna = False)

$682.00    1
$249.00    1
$242.00    1
$634.99    1
$949.99    1
$303.00    1
$588.00    1
$219.99    1
$339.00    1
$579.99    1
$638.99    1
$249.43    1
$355.98    1
Name: Preco, dtype: int64

Ok. A coluna preço está com o símbolo de $ nos valores. Vamos remover isso e ajustar pontos e vírgulas.

In [22]:
# Função para limpeza da coluna de preço
def limpa_preco(price):
    
    # Remove o '$'
    price = price.replace('$', '')  
    
    # Remove a ',' 
    price = price.replace(',', '') 
    
    # Converte o tipo de dado para numérico
    price = pd.to_numeric(price)   
    
    # Arredonda os valores
    price = price.round()   
    
    return price

In [23]:
# Criamos uma nova coluna com o preço limpo
df_dados_notebooks['Preco_Final'] = df_dados_notebooks['Preco'].apply(limpa_preco)

In [24]:
# Vamos remover os registros com preços menores que 150, pois devem ser acessórios
df_dados_notebooks = df_dados_notebooks[df_dados_notebooks['Preco_Final'] >= 150]

In [25]:
# Shape
df_dados_notebooks.shape

(13, 4)

In [26]:
# Visualiza os dados
df_dados_notebooks.head(10)

,Nome,Preco,Avaliacao,Preco_Final
0,"2020 HP Pavilion 14"" HD Display Laptop Compute...",$339.00,4.2 out of 5 stars,339.0
1,HP 14in High Performance Laptop (AMD Ryzen 3 3...,$303.00,3.8 out of 5 stars,303.0
2,"ASUS ZenBook 13 Ultra Slim Laptop, 13.3” FHD W...",$949.99,4.0 out of 5 stars,950.0
3,2020 Lenovo IdeaPad Laptop ComputerAMD A6-9220...,$242.00,3.5 out of 5 stars,242.0
6,2018 Newest HP Premium High Performance Busine...,$355.98,4.4 out of 5 stars,356.0
7,"2020 HP 17.3"" Laptop Computer/ 8th Gen Intel Q...",$588.00,4.2 out of 5 stars,588.0
8,"New HP Pavilion 2-in-1 15.6"" HD Touchscreen La...",$682.00,4.3 out of 5 stars,682.0
9,"HP Stream 14-inch Laptop, AMD Dual-Core A4-912...",$219.99,4.1 out of 5 stars,220.0
10,"Jumper EZbook X3 Windows 10 Laptop, Laptop com...",$249.00,3.8 out of 5 stars,249.0
11,"HP Stream 14-inch Laptop, AMD Dual-Core A4-912...",$249.43,4.0 out of 5 stars,249.0


In [27]:
# Não precisamos mais da coluna antiga. Vamos removê-la
df_dados_notebooks.drop('Preco', axis = 1, inplace = True)

In [28]:
# Visualiza os dados
df_dados_notebooks.head(3)

,Nome,Avaliacao,Preco_Final
0,"2020 HP Pavilion 14"" HD Display Laptop Compute...",4.2 out of 5 stars,339.0
1,HP 14in High Performance Laptop (AMD Ryzen 3 3...,3.8 out of 5 stars,303.0
2,"ASUS ZenBook 13 Ultra Slim Laptop, 13.3” FHD W...",4.0 out of 5 stars,950.0


## Limpeza da Coluna de Avaliações

A coluna de avaliações possui valores como **3.8 out of 5 stars**. O que é importante aqui para a nossa análise é o primeiro valor. Realizaremos as operações de limpeza nessa coluna.

In [29]:
# Número de elementos por tipo de avaliação
df_dados_notebooks['Avaliacao'].value_counts(dropna = False)

4.3 out of 5 stars    3
4.2 out of 5 stars    3
4.0 out of 5 stars    2
3.8 out of 5 stars    2
4.4 out of 5 stars    1
3.5 out of 5 stars    1
4.1 out of 5 stars    1
Name: Avaliacao, dtype: int64

In [31]:
# Função para limpar as avaliações e remover no-info
def limpa_aval(aval):
    
    # Divide a avaliação por espaços
    aval = aval.split()
    
    # Extraímos a primeira coluna (de índice zero)
    aval = aval[0]
    
    # Verificamos o valor 
    if aval == 'no-info':
        aval = 0
        
    # Convertemos para o tipo inteiro    
    aval = pd.to_numeric(aval) 
    
    return aval

In [30]:
# Criamos uma nova coluna com o valor processado
df_dados_notebooks['Avaliacao_Final'] = df_dados_notebooks['Avaliacao'].apply(limpa_aval)

In [31]:
# Shape
df_dados_notebooks.shape

(54, 4)

In [32]:
# Visualiza os dados
df_dados_notebooks.head(10)

,Nome,Avaliacao,Preco_Final,Avaliacao_Final
0,"ASUS VivoBook 15 Thin and Light Laptop, 15.6” ...",4.3 out of 5 stars,400.0,4.3
1,"Lenovo IdeaPad 3 14"" Laptop, 14.0"" FHD (1920 x...",3.8 out of 5 stars,450.0,3.8
2,(Renewed) Acer Flagship CB3-532 15.6 inches HD...,4.2 out of 5 stars,198.0,4.2
3,"New ! Dell Inspiron i3583 15.6"" HD Touch-Scree...",4.4 out of 5 stars,597.0,4.4
7,"Lenovo Ideapad L340 Gaming Laptop, 15.6 Inch F...",4.4 out of 5 stars,858.0,4.4
9,Razer Blade 15 Gaming Laptop: Intel Core i7-97...,4.1 out of 5 stars,1346.0,4.1
10,CHUWI HeroBook Pro 14.1 inch Windows 10 Laptop...,3.5 out of 5 stars,279.0,3.5
11,"ALLDOCUBE KBook Lite 2-in-1 Laptop, 13.5 inch ...",no-info,280.0,0.0
12,2020 Lenovo IdeaPad 14 Inch Laptop| AMD A6-922...,3.7 out of 5 stars,279.0,3.7
13,"HP 14"" Touchscreen Home and Business Laptop Ry...",4.3 out of 5 stars,473.0,4.3


In [33]:
# Não precisamos mais da coluna antiga. Vamos removê-la
df_dados_notebooks.drop('Avaliacao', axis = 1, inplace = True)

In [34]:
# Visualiza os dados
df_dados_notebooks.head(3)

,Nome,Preco_Final,Avaliacao_Final
0,"ASUS VivoBook 15 Thin and Light Laptop, 15.6” ...",400.0,4.3
1,"Lenovo IdeaPad 3 14"" Laptop, 14.0"" FHD (1920 x...",450.0,3.8
2,(Renewed) Acer Flagship CB3-532 15.6 inches HD...,198.0,4.2


Muito bom. Estamos quase lá. Vamos agora processar a coluna nome.

## Limpeza da Coluna Nome

A coluna de nome consiste em muitas informações sobre o produto, além de detalhes técnicos, como nome da marca, tamanho da tela, memória e assim por diante. Portanto, para fazer uma análise eficaz, precisamos extrair essas informações. Nosso próximo passo consiste nesse processo. Esta é a parte mais complicada deste Lab. Para extração, precisamos usar expressões regulares.

In [35]:
# Vamos visualizar a coluna Nome
df_dados_notebooks['Nome']

0     ASUS VivoBook 15 Thin and Light Laptop, 15.6” ...
1     Lenovo IdeaPad 3 14" Laptop, 14.0" FHD (1920 x...
2     (Renewed) Acer Flagship CB3-532 15.6 inches HD...
3     New ! Dell Inspiron i3583 15.6" HD Touch-Scree...
7     Lenovo Ideapad L340 Gaming Laptop, 15.6 Inch F...
9     Razer Blade 15 Gaming Laptop: Intel Core i7-97...
10    CHUWI HeroBook Pro 14.1 inch Windows 10 Laptop...
11    ALLDOCUBE KBook Lite 2-in-1 Laptop, 13.5 inch ...
12    2020 Lenovo IdeaPad 14 Inch Laptop| AMD A6-922...
13    HP 14" Touchscreen Home and Business Laptop Ry...
14    HP Stream 14-inch Laptop, AMD Dual-Core A4-912...
15    2020 Lenovo IdeaPad Laptop ComputerAMD A6-9220...
17    Lenovo Ideapad 330 15.6" Anti Glared HD Premiu...
18    New Lenovo Ideapad S145 14" Laptop Intel Penti...
19    2020 ASUS VivoBook 15 15.6 Inch FHD 1080P Lapt...
20    Dell Latitude 14 7000 7480 Business UltraBook ...
21    IDOL Laptop 10.1Inch Quad Core Ultra Thin PC, ...
22    Dell Latitude 14 7000 7480 Business UltraB

Esses 3 pontinhos ao final indicam que a string é muito longa e por isso foi truncada pelo pandas. Mas somos resolvedores de problemas, pesquisamos na documentação Python e encontramos como resolver isso.

In [36]:
# Parâmetro para imprimir nomes completos
pd.set_option('display.max_colwidth', None)

In [37]:
# Vamos visualizar a coluna Nome
df_dados_notebooks['Nome']

0                    ASUS VivoBook 15 Thin and Light Laptop, 15.6” FHD Display, Intel i3-1005G1 CPU, 8GB RAM, 128GB SSD, Backlit Keyboard, Fingerprint, Windows 10 Home in S Mode, Slate Gray, F512JA-AS34
1         Lenovo IdeaPad 3 14" Laptop, 14.0" FHD (1920 x 1080) Display, AMD Ryzen 5 3500U Processor, 8GB DDR4 RAM, 256GB SSD, AMD Radeon Vega 8 Graphics, Narrow Bezel, Windows 10, 81W0003QUS, Abyss Blue
2                       (Renewed) Acer Flagship CB3-532 15.6 inches HD Premium Chromebook - Intel Dual-Core Celeron N3060 up to 2.48GH.z, 2GB RAM, 16GB SSD, Wireless AC, HDMI, USB 3.0, Webcam, Chrome OS
3          New ! Dell Inspiron i3583 15.6" HD Touch-Screen Laptop - Intel i5-8265U - 8GB DDR4-256GB SSD - Windows 10 - Wireless-AC - Bluetooth, SD Card Reader, HDMI & USB 3.1 -Waves MaxxAudio Pro- Black
7          Lenovo Ideapad L340 Gaming Laptop, 15.6 Inch FHD (1920 X 1080) IPS Display, Intel Core i5-9300H Processor, 8GB DDR4 RAM, 512GB Nvme SSD, NVIDIA GeForce GTX 1650, Windows 10, 81L

In [38]:
# Vamos visualizar um registro
df_dados_notebooks['Nome'][3]

'New ! Dell Inspiron i3583 15.6" HD Touch-Screen Laptop - Intel i5-8265U - 8GB DDR4-256GB SSD - Windows 10 - Wireless-AC - Bluetooth, SD Card Reader, HDMI & USB 3.1 -Waves MaxxAudio Pro- Black'

Agora sim! Primeiro, extrairemos os detalhes do processador e criaremos uma nova coluna com essas informações. Se não compreender a regra usada abaixo com expressão regular, então visite a documentação oficial:

https://docs.python.org/3/library/re.html

In [39]:
# Regra para buscar o padrão na string
regra_busca_processador = r'\b([iI][\d])\b'

In [40]:
# Cria a coluna para o processador
df_dados_notebooks['Processador'] = df_dados_notebooks['Nome'].str.extract(regra_busca_processador)

Vamos padronizar o nome do processador substituindo I5 por i5, por exemplo.

In [41]:
# Replace 'I5' com 'i5'
df_dados_notebooks['Processador'] = df_dados_notebooks['Processador'].str.replace('I', 'i')

In [42]:
# Visualiza os dados
df_dados_notebooks.head()

,Nome,Preco_Final,Avaliacao_Final,Processador
0,"ASUS VivoBook 15 Thin and Light Laptop, 15.6” FHD Display, Intel i3-1005G1 CPU, 8GB RAM, 128GB SSD, Backlit Keyboard, Fingerprint, Windows 10 Home in S Mode, Slate Gray, F512JA-AS34",400.0,4.3,i3
1,"Lenovo IdeaPad 3 14"" Laptop, 14.0"" FHD (1920 x 1080) Display, AMD Ryzen 5 3500U Processor, 8GB DDR4 RAM, 256GB SSD, AMD Radeon Vega 8 Graphics, Narrow Bezel, Windows 10, 81W0003QUS, Abyss Blue",450.0,3.8,NaN
2,"(Renewed) Acer Flagship CB3-532 15.6 inches HD Premium Chromebook - Intel Dual-Core Celeron N3060 up to 2.48GH.z, 2GB RAM, 16GB SSD, Wireless AC, HDMI, USB 3.0, Webcam, Chrome OS",198.0,4.2,NaN
3,"New ! Dell Inspiron i3583 15.6"" HD Touch-Screen Laptop - Intel i5-8265U - 8GB DDR4-256GB SSD - Windows 10 - Wireless-AC - Bluetooth, SD Card Reader, HDMI & USB 3.1 -Waves MaxxAudio Pro- Black",597.0,4.4,i5
7,"Lenovo Ideapad L340 Gaming Laptop, 15.6 Inch FHD (1920 X 1080) IPS Display, Intel Core i5-9300H Processor, 8GB DDR4 RAM, 512GB Nvme SSD, NVIDIA GeForce GTX 1650, Windows 10, 81LK00HDUS, Black",858.0,4.4,i5


Valores nulos foram gerados, provavelmente porque alguns laptops devem ter outro tipo de processador. Mas buscaremos somente core i5 ou i7. 

Usaremos o mesmo processo para extrair informações sobre a memória RAM.

In [43]:
# Regra de extração   
regra_busca_ram = r'\b([\d]+)[GB]+[ ][\+ LlPpDdRrAaMmEeOoYy\d]+\b'

In [44]:
# Extrai os dados e cria nova coluna
df_dados_notebooks['Memoria_RAM'] = df_dados_notebooks['Nome'].str.extract(regra_busca_ram)

In [45]:
# Função para limpar a coluna Memoria_RAM
def limpa_ram(ram):
    ram = pd.to_numeric(ram)
    return ram

In [46]:
# Aplica a função de limpeza
df_dados_notebooks['Memoria_RAM'] = df_dados_notebooks['Memoria_RAM'].apply(limpa_ram)

In [47]:
# Visualiza os dados
df_dados_notebooks.head()

,Nome,Preco_Final,Avaliacao_Final,Processador,Memoria_RAM
0,"ASUS VivoBook 15 Thin and Light Laptop, 15.6” FHD Display, Intel i3-1005G1 CPU, 8GB RAM, 128GB SSD, Backlit Keyboard, Fingerprint, Windows 10 Home in S Mode, Slate Gray, F512JA-AS34",400.0,4.3,i3,8.0
1,"Lenovo IdeaPad 3 14"" Laptop, 14.0"" FHD (1920 x 1080) Display, AMD Ryzen 5 3500U Processor, 8GB DDR4 RAM, 256GB SSD, AMD Radeon Vega 8 Graphics, Narrow Bezel, Windows 10, 81W0003QUS, Abyss Blue",450.0,3.8,NaN,8.0
2,"(Renewed) Acer Flagship CB3-532 15.6 inches HD Premium Chromebook - Intel Dual-Core Celeron N3060 up to 2.48GH.z, 2GB RAM, 16GB SSD, Wireless AC, HDMI, USB 3.0, Webcam, Chrome OS",198.0,4.2,NaN,2.0
3,"New ! Dell Inspiron i3583 15.6"" HD Touch-Screen Laptop - Intel i5-8265U - 8GB DDR4-256GB SSD - Windows 10 - Wireless-AC - Bluetooth, SD Card Reader, HDMI & USB 3.1 -Waves MaxxAudio Pro- Black",597.0,4.4,i5,8.0
7,"Lenovo Ideapad L340 Gaming Laptop, 15.6 Inch FHD (1920 X 1080) IPS Display, Intel Core i5-9300H Processor, 8GB DDR4 RAM, 512GB Nvme SSD, NVIDIA GeForce GTX 1650, Windows 10, 81LK00HDUS, Black",858.0,4.4,i5,8.0


Por fim, vamos extrair o tamanho da tela do notebook.

In [48]:
# Regra de extração
regra_busca_tela = r'\b([1][\d]+[\.IiNnCcHh\d]*)[ \d]*\b' 

In [49]:
df_dados_notebooks['Tamanho_Tela'] = df_dados_notebooks['Nome'].str.extract(regra_busca_tela)

In [50]:
# Buscando o total de registros por Tamanho_Tela
df_dados_notebooks['Tamanho_Tela'].value_counts(dropna = False)

14          18
15.6         9
15           7
14.1         4
10.1Inch     3
13           3
11.6         3
11.6in       1
10           1
13.5         1
11.6inch     1
17           1
17.3         1
1769         1
Name: Tamanho_Tela, dtype: int64

In [51]:
# Função de limpeza
def limpa_tela(size):
    size = size.replace('Inch', '')
    size = size.replace('inch', '')
    size = size.replace('in', '')
    size = pd.to_numeric(size)
    return size

In [52]:
# Aplica a função
df_dados_notebooks['Tamanho_Tela'] = df_dados_notebooks['Tamanho_Tela'].apply(limpa_tela)

In [53]:
# Shape
df_dados_notebooks.shape

(54, 6)

In [54]:
# Visualiza os dados
df_dados_notebooks.head()

,Nome,Preco_Final,Avaliacao_Final,Processador,Memoria_RAM,Tamanho_Tela
0,"ASUS VivoBook 15 Thin and Light Laptop, 15.6” FHD Display, Intel i3-1005G1 CPU, 8GB RAM, 128GB SSD, Backlit Keyboard, Fingerprint, Windows 10 Home in S Mode, Slate Gray, F512JA-AS34",400.0,4.3,i3,8.0,15.0
1,"Lenovo IdeaPad 3 14"" Laptop, 14.0"" FHD (1920 x 1080) Display, AMD Ryzen 5 3500U Processor, 8GB DDR4 RAM, 256GB SSD, AMD Radeon Vega 8 Graphics, Narrow Bezel, Windows 10, 81W0003QUS, Abyss Blue",450.0,3.8,NaN,8.0,14.0
2,"(Renewed) Acer Flagship CB3-532 15.6 inches HD Premium Chromebook - Intel Dual-Core Celeron N3060 up to 2.48GH.z, 2GB RAM, 16GB SSD, Wireless AC, HDMI, USB 3.0, Webcam, Chrome OS",198.0,4.2,NaN,2.0,15.6
3,"New ! Dell Inspiron i3583 15.6"" HD Touch-Screen Laptop - Intel i5-8265U - 8GB DDR4-256GB SSD - Windows 10 - Wireless-AC - Bluetooth, SD Card Reader, HDMI & USB 3.1 -Waves MaxxAudio Pro- Black",597.0,4.4,i5,8.0,15.6
7,"Lenovo Ideapad L340 Gaming Laptop, 15.6 Inch FHD (1920 X 1080) IPS Display, Intel Core i5-9300H Processor, 8GB DDR4 RAM, 512GB Nvme SSD, NVIDIA GeForce GTX 1650, Windows 10, 81LK00HDUS, Black",858.0,4.4,i5,8.0,15.6


## Conclusão

Estamos prontos para realizar nossa análise e concluir o Lab.

In [55]:
# Nomes das colunas finais do Dataset
nomes_colunas = ['Nome', 'Preco_Final', 'Avaliacao_Final', 'Processador', 'Memoria_RAM', 'Tamanho_Tela']

Vamos filtrar os resultados e retornar máquinas com processador i5 ou i7, 16 GB de RAM ou mais, tamanho da tela de 14 ou 15 poleadas e avaliação igual a 4. Fique à vontade para mudar as regras se desejar.

In [56]:
# Dicionário para os filtros
filtros = {'processador':{'min':'i5', 'max':'i7'}, 
           'ram':16, 
           'preco':1500, 
           'tela':{'min':14, 'max':15}, 
           'aval':4}

In [57]:
# Aplicamos os filtros para a coluna do processador
df_dados_notebooks = df_dados_notebooks[nomes_colunas][(df_dados_notebooks['Processador'] == filtros['processador']['min']) | (df_dados_notebooks['Processador'] == filtros['processador']['max'])].copy()


In [58]:
# Aplicamos os filtros para a coluna de memória RAM e preço
df_dados_notebooks = df_dados_notebooks[(df_dados_notebooks['Memoria_RAM'] >= filtros['ram']) & (df_dados_notebooks['Preco_Final'] <= filtros['preco'])]

In [59]:
# Aplicamos os filtros para a coluna de tamanho de tela
df_dados_notebooks = df_dados_notebooks[(df_dados_notebooks['Tamanho_Tela'] >= filtros['tela']['min']) & (df_dados_notebooks['Tamanho_Tela'] <= filtros['tela']['max'])]


In [60]:
# Por fim, filtramos as avaliações
df_dados_notebooks = df_dados_notebooks[df_dados_notebooks['Avaliacao_Final'] >= filtros['aval']]

Podemos então visualizar o resultado do nosso trabalho.

In [61]:
# Shape
df_dados_notebooks.shape

(1, 6)

In [62]:
# Resultado final
df_dados_notebooks

,Nome,Preco_Final,Avaliacao_Final,Processador,Memoria_RAM,Tamanho_Tela
9,"Razer Blade 15 Gaming Laptop: Intel Core i7-9750H 6 Core, NVIDIA GeForce GTX 1660 Ti, 15.6"" FHD 144Hz, 16GB RAM, 256GB SSD, CNC Aluminum, Chroma RGB Lighting, Thunderbolt 3, 9th Gen i7-9750H",1346.0,4.1,i7,16.0,15.0


E aí está o resultado final de acordo com os filtros que escolhemos. Como os produtos mudam constantemente no site da Amazon, cada execução pode ser diferente.

Você poderia ainda extrair detalhes do disco, GPU, geração do processador, etc... 

Trabalho concluído. Tente agora extrair os dados do portal brasileiro da Amazon ou mesmo de outros sites de venda de notebooks.

# Fim